In [1]:
from pathlib import Path
from os.path import expanduser
from os import path
import numpy as np
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models
from torchsummary import summary
from torchvision import transforms, datasets
from torch.optim import lr_scheduler
import os

In [2]:
root_folder = os.path.dirname(os.getcwd())
path_to_train_data = path.join(root_folder,'data/train_data.npy');
path_to_labels = path.join(root_folder,'data/train_labels.npy');
path_to_mean_std = path.join(root_folder,'data/mean_std.npy');

sys.path.append(root_folder)
from custom_resnet import CustomResnet as cnn

In [3]:
num_classes = 2;
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size = 15000;
nb_of_epochs = 25;
logging_interval = 30
dataset_divider = 0.2
net = cnn.resnet18(num_classes = num_classes);
net.to(device)

ResNet(
  (conv1): Conv1d(1, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv1d(64, 64, kernel_size

In [4]:
data = np.load(path_to_train_data);
labels = np.load(path_to_labels);
torch_data = torch.tensor(data).float()
torch_labels = torch.tensor(labels).int()
train_data, test_data = cnn.GenerateTrainAndTestDataset(torch_data, torch_labels, 0.2)


In [5]:
mean_std = torch.tensor(np.load(path_to_mean_std))
mean_train = mean_std[0].repeat(train_data.__len__());
std_train = mean_std[1].repeat(train_data.__len__());
mean_test = mean_std[0].repeat(test_data.__len__());
std_test = mean_std[1].repeat(test_data.__len__());
transform = transforms.Compose([transforms.Normalize(mean_train, std_train)])
train_data.data = transform(train_data.data)
transform = transforms.Compose([transforms.Normalize(mean_test, std_test)])
test_data.data = transform(test_data.data)

/home/vtpc/.local/lib/python3.5/site-packages/torchvision/transforms/functional.py:206: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mean = torch.tensor(mean, dtype=torch.float32)
/home/vtpc/.local/lib/python3.5/site-packages/torchvision/transforms/functional.py:207: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  std = torch.tensor(std, dtype=torch.float32)


In [6]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=8)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=8)




In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
for i in range(nb_of_epochs):  # loop over the dataset multiple times
  loss = cnn.Train(net, device, train_loader, optimizer, criterion, exp_lr_scheduler, i, logging_interval)
  model_name = 'models/resnet18_num_classes_' + str(num_classes) + "_epoch_" + str(i) + '.pt';
  path_to_model = path.join(root_folder, model_name)
  torch.save({'epoch': i,'model_state_dict': net.state_dict(), 'optimizer_state_dict': optimizer.state_dict(), 'loss': loss,}, path_to_model)
  cnn.Test(net, device, criterion, test_loader)

Train Epoch: 0 [0/9779124 (0%)]	Loss: 0.66788178682327270508
Train Epoch: 0 [450000/9779124 (5%)]	Loss: 0.11163564771413803101
Train Epoch: 0 [900000/9779124 (9%)]	Loss: 0.04925263300538063049
Train Epoch: 0 [1350000/9779124 (14%)]	Loss: 0.03379075974225997925
Train Epoch: 0 [1800000/9779124 (18%)]	Loss: 0.02545981481671333313
Train Epoch: 0 [2250000/9779124 (23%)]	Loss: 0.02065239474177360535
Train Epoch: 0 [2700000/9779124 (28%)]	Loss: 0.01644353196024894714
Train Epoch: 0 [3150000/9779124 (32%)]	Loss: 0.01423525251448154449
Train Epoch: 0 [3600000/9779124 (37%)]	Loss: 0.01315480656921863556
Train Epoch: 0 [4050000/9779124 (41%)]	Loss: 0.01094729825854301453
Train Epoch: 0 [4500000/9779124 (46%)]	Loss: 0.01036600302904844284
Train Epoch: 0 [4950000/9779124 (51%)]	Loss: 0.00981261022388935089
Train Epoch: 0 [5400000/9779124 (55%)]	Loss: 0.00937879551202058792
Train Epoch: 0 [5850000/9779124 (60%)]	Loss: 0.00834388844668865204
Train Epoch: 0 [6300000/9779124 (64%)]	Loss: 0.007979733869

Train Epoch: 5 [1800000/9779124 (18%)]	Loss: 0.00268799229525029659
Train Epoch: 5 [2250000/9779124 (23%)]	Loss: 0.00121528760064393282
Train Epoch: 5 [2700000/9779124 (28%)]	Loss: 0.00168574287090450525
Train Epoch: 5 [3150000/9779124 (32%)]	Loss: 0.00258571072481572628
Train Epoch: 5 [3600000/9779124 (37%)]	Loss: 0.00162243982776999474
Train Epoch: 5 [4050000/9779124 (41%)]	Loss: 0.00206401548348367214
Train Epoch: 5 [4500000/9779124 (46%)]	Loss: 0.00164207257330417633
Train Epoch: 5 [4950000/9779124 (51%)]	Loss: 0.00312307337298989296
Train Epoch: 5 [5400000/9779124 (55%)]	Loss: 0.00156594451982527971
Train Epoch: 5 [5850000/9779124 (60%)]	Loss: 0.00108316261321306229
Train Epoch: 5 [6300000/9779124 (64%)]	Loss: 0.00181115325540304184
Train Epoch: 5 [6750000/9779124 (69%)]	Loss: 0.00154946907423436642
Train Epoch: 5 [7200000/9779124 (74%)]	Loss: 0.00221530534327030182
Train Epoch: 5 [7650000/9779124 (78%)]	Loss: 0.00259904353879392147
Train Epoch: 5 [8100000/9779124 (83%)]	Loss: 0.0

Train Epoch: 10 [3600000/9779124 (37%)]	Loss: 0.00151217705570161343
Train Epoch: 10 [4050000/9779124 (41%)]	Loss: 0.00185826595406979322
Train Epoch: 10 [4500000/9779124 (46%)]	Loss: 0.00128892587963491678
Train Epoch: 10 [4950000/9779124 (51%)]	Loss: 0.00320690753869712353
Train Epoch: 10 [5400000/9779124 (55%)]	Loss: 0.00142886757384985685
Train Epoch: 10 [5850000/9779124 (60%)]	Loss: 0.00279301707632839680
Train Epoch: 10 [6300000/9779124 (64%)]	Loss: 0.00146773434244096279
Train Epoch: 10 [6750000/9779124 (69%)]	Loss: 0.00277788634411990643
Train Epoch: 10 [7200000/9779124 (74%)]	Loss: 0.00164683861657977104
Train Epoch: 10 [7650000/9779124 (78%)]	Loss: 0.00176976143848150969
Train Epoch: 10 [8100000/9779124 (83%)]	Loss: 0.00192952097859233618
Train Epoch: 10 [8550000/9779124 (87%)]	Loss: 0.00172313011717051268
Train Epoch: 10 [9000000/9779124 (92%)]	Loss: 0.00257909018546342850
Train Epoch: 10 [9450000/9779124 (97%)]	Loss: 0.00214459467679262161

Test set: Average loss: 0.0000000

Train Epoch: 15 [4950000/9779124 (51%)]	Loss: 0.00146374269388616085
Train Epoch: 15 [5400000/9779124 (55%)]	Loss: 0.00241169007495045662
Train Epoch: 15 [5850000/9779124 (60%)]	Loss: 0.00164273427799344063
Train Epoch: 15 [6300000/9779124 (64%)]	Loss: 0.00191043736413121223
Train Epoch: 15 [6750000/9779124 (69%)]	Loss: 0.00207209330983459949
Train Epoch: 15 [7200000/9779124 (74%)]	Loss: 0.00157828791998326778
Train Epoch: 15 [7650000/9779124 (78%)]	Loss: 0.00200155214406549931
Train Epoch: 15 [8100000/9779124 (83%)]	Loss: 0.00233318051323294640
Train Epoch: 15 [8550000/9779124 (87%)]	Loss: 0.00199472508393228054
Train Epoch: 15 [9000000/9779124 (92%)]	Loss: 0.00157737359404563904
Train Epoch: 15 [9450000/9779124 (97%)]	Loss: 0.00189694005530327559

Test set: Average loss: 0.00000009193658456752, Accuracy: 2444113/2444780 (99.97271738152308273584%)

Train Epoch: 16 [0/9779124 (0%)]	Loss: 0.00157169380690902472
Train Epoch: 16 [450000/9779124 (5%)]	Loss: 0.00218443665653467178
Train Epo

Train Epoch: 20 [6300000/9779124 (64%)]	Loss: 0.00186622608453035355
Train Epoch: 20 [6750000/9779124 (69%)]	Loss: 0.00220393319614231586
Train Epoch: 20 [7200000/9779124 (74%)]	Loss: 0.00173428608104586601
Train Epoch: 20 [7650000/9779124 (78%)]	Loss: 0.00125295517500489950
Train Epoch: 20 [8100000/9779124 (83%)]	Loss: 0.00164069992024451494
Train Epoch: 20 [8550000/9779124 (87%)]	Loss: 0.00199247151613235474
Train Epoch: 20 [9000000/9779124 (92%)]	Loss: 0.00197541690431535244
Train Epoch: 20 [9450000/9779124 (97%)]	Loss: 0.00129935774020850658

Test set: Average loss: 0.00000009193954042530, Accuracy: 2444117/2444780 (99.97288099542699058020%)

Train Epoch: 21 [0/9779124 (0%)]	Loss: 0.00150778389070183039
Train Epoch: 21 [450000/9779124 (5%)]	Loss: 0.00216927635483443737
Train Epoch: 21 [900000/9779124 (9%)]	Loss: 0.00273085990920662880
Train Epoch: 21 [1350000/9779124 (14%)]	Loss: 0.00166702817659825087
Train Epoch: 21 [1800000/9779124 (18%)]	Loss: 0.00186449277680367231
Train Epoch

In [8]:
cnn.Test(net, device, criterion, test_loader)


Test set: Average loss: 0.00000009237253095762, Accuracy: 2444088/2444780 (99.97169479462364449773%)

